# American Put Options with the Binomial Asset Pricing Model


Implementation of the simple slow binomial pricing model in python. We will treat binomial tree as a network with nodes (i,j) with i representing the time steps and j representing the number of ordered price outcome (lowest - or bottom of tree - to highest).

- american_tree



In [12]:
import numpy as np

### Generic timing wrapper function
We will use this to benchmark the two binomial models

In [13]:
from functools import wraps
from time import time

def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        print('func:%r args:[%r, %r] took: %2.4f sec' % \
          (f.__name__, args, kw, te-ts))
        return result
    return wrap

### Binomial Tree Representation

Stock tree can be represented using nodes (i,j) and intial stock price $S_0$

$S_{i,j} = S_0u^{j}d^{i-j}$

$C_{i,j}$ represents contract price at each node (i,j). Where $C_{N,j}$ represents final payoff function that we can define.

### American Option Characteristics
For an American Put Option:

if $T = t_N$ then at the terminal nodes, $C^{j}_N = (K-S^{j}_N)^{+}$

for all other parts of the tree at nodes $(i,j)$

- Max of exercise value or continuation/hold value

- $C^{j}_i = \max \Big((K-S^{j}_i)^{+}, \exp^{-r\Delta t} \big\{ q^{j}_i C^{j+1}_{i+1} + (1 - q^{j}_i)C^{j-1}_{i-1}\big\}\Big)$

In [14]:
# Initialise parameters
S0 = 100      # initial stock price
K = 100       # strike price
T = 1         # time to maturity in years
r = 0.06      # annual risk-free rate
N = 3         # number of time steps
u = 1.1       # up-factor in binomial models
d = 0.9    # ensure recombining tree
opttype = 'C' # Option Type 'C' or 'P'

### American Tree Slow

Here we will use for loops to iterate through nodes j at each time step i.

In [15]:
 @timing
def american_tree(K,T,S0,r,N,u,d,opttype='C'):
    #precompute values
    dt = T/N
    q = (np.exp(r*dt) - d)/(u-d)
    disc = np.exp(-r*dt)

    # initialise stock prices at maturity
    S = np.zeros(N+1)
    for j in range(0, N+1):
        S[j] = S0 * u**j * d**(N-j)

    # option payoff
    C = np.zeros(N+1)
    for j in range(0, N+1):
        if opttype == 'P':
            C[j] = max(0, K - S[j])
        else:
            C[j] = max(0, S[j] - K)

    # backward recursion through the tree
    for i in np.arange(N-1,-1,-1):
        for j in range(0,i+1):
            S = S0 * u**j * d**(i-j)
            C[j] = disc * ( q*C[j+1] + (1-q)*C[j] )
            if opttype == 'P':
                C[j] = max(C[j], K - S)
            else:
                C[j] = max(C[j], S - K)

    return C[0]

american_tree(K,T,S0,r,N,u,d,opttype='C')

func:'american_tree' args:[(100, 1, 100, 0.06, 3, 1.1, 0.9), {'opttype': 'C'}] took: 0.0010 sec


10.391101869597515

### Binomial Tree Slow Times

Now we will compare runtimes for the algo 

In [16]:
for N in [3,50, 100, 1000, 5000]:
    american_tree(K,T,S0,r,N,u,d,opttype='C')

func:'american_tree' args:[(100, 1, 100, 0.06, 3, 1.1, 0.9), {'opttype': 'C'}] took: 0.0000 sec
func:'american_tree' args:[(100, 1, 100, 0.06, 50, 1.1, 0.9), {'opttype': 'C'}] took: 0.0040 sec
func:'american_tree' args:[(100, 1, 100, 0.06, 100, 1.1, 0.9), {'opttype': 'C'}] took: 0.0150 sec
func:'american_tree' args:[(100, 1, 100, 0.06, 1000, 1.1, 0.9), {'opttype': 'C'}] took: 1.5360 sec
func:'american_tree' args:[(100, 1, 100, 0.06, 5000, 1.1, 0.9), {'opttype': 'C'}] took: 37.7728 sec
